In [1]:
import cv2
import numpy as np
import math

def compute_edge_image(bgr_img):
    """ Compute the edge magnitude of an image using a pair of Sobel filters """

    sobel_v = np.array([[-1, -2, -1],   # Sobel filter for the vertical gradient. Note that the filter2D function computes a correlation
                        [ 0,  0, 0],    # instead of a convolution, so the filter is *not* rotated by 180 degrees.
                        [ 1,  2, 1]])
    sobel_h = sobel_v.T                 # The convolution filter for the horizontal gradient is simply the transpose of the previous one  

    gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)
    gradient_v = cv2.filter2D(gray_img, ddepth=cv2.CV_32F, kernel=sobel_v)
    gradient_h = cv2.filter2D(gray_img, ddepth=cv2.CV_32F, kernel=sobel_h)
    gradient_magni = np.sqrt(gradient_v**2 + gradient_h**2)
    near_max = np.percentile(gradient_magni, 99.5)      # Clip magnitude at percentile 99.5 to prevent outliers from determining the range of relevant magnitudes  
    edge_img = np.clip(gradient_magni * 255.0 / near_max, 0.0, 255.0).astype(np.uint8)  # Normalize and convert magnitudes into grayscale image 
    return edge_img

In [2]:
# def hough_transform_lines(input_space, num_alpha_bins, num_d_bins):
#     """" Perform Hough transform of an image to an alpha-d space represemnting straight lines """

#     output_space = np.zeros((num_alpha_bins, num_d_bins), dtype=np.int)

#     # Generate matrix of sine and cosine values for each alpha bin to speed up subsequent computation.
#     d_max = math.sqrt(input_space.shape[0]**2 + input_space.shape[1]**2)
#     alpha_bins = np.linspace(-0.5 * math.pi, math.pi, num_alpha_bins) # Value of angle alpha for each bin, i.e., row in the output matrix
#     cos_sin_matrix = np.column_stack((np.cos(alpha_bins), np.sin(alpha_bins))) * num_d_bins / d_max     # cos and sin values of alpha for each bin (normalized)

#     edge_coords = np.row_stack(np.nonzero(input_space >= 64))   # Only consider edges exceeding a threshold; higher threshold speeds up computation but ignores more edges (can also go to 128 --> higher)
#     edge_magnitudes = input_space[edge_coords[0, :], edge_coords[1, :]]     # List of magnitudes of all considered edges 
#     d_bins = np.matmul(cos_sin_matrix, edge_coords).astype(np.int)          # Matrix multiplication yields matrix of d-values of shape num_alpha_bins x number of considered edges
    
#     for alpha_bin in range(num_alpha_bins):
#         for edge_index in range(len(edge_magnitudes)):
#             d_bin = d_bins[alpha_bin, edge_index]
#             if d_bin >= 0.5:                                                    # For each considered edge and value of alpha, if d is positive,
#                 output_space[alpha_bin, d_bin] += edge_magnitudes[edge_index]   # increase the current (alpha, d) counter by the edge magnitude
    
#     return output_space, alpha_bins, d_max

# def find_hough_maxima(output_space, num_maxima, min_dist_alpha, min_dist_d):
#     """ Find the given number of vote maxima in the output space with certain minimum distances in alpha and d between them """
#     maxima = []
#     output_copy = output_space.copy()
#     height, width = output_copy.shape
    
#     for i in range(num_maxima):
#         row, col = np.unravel_index(np.argmax(output_copy), (height, width))    # Get coordinates (alpha, d) of global maximum
#         maxima.append((row, col))
#         output_copy[max(0, row - min_dist_alpha):min(height - 1, row + min_dist_alpha),     # Set all cells within the minimum distances to -1
#                     max(0, col - min_dist_d):    min(width - 1,  col + min_dist_d)] = -1.0  # so that no further maxima will be selected from this area
    
#     return maxima       # Return list of (alpha, d) pairs indicating locations of maxima 

# def draw_hough_line(img, alpha, d):
#     """ Add a straight line with pa rameters alpha (radians) and d (pixels) to a given image """

#     h, w = img.shape[:2]
#     i0, j0 = d * math.cos(alpha), d * math.sin(alpha)   # Determine (i0, j0) where normal intersects with line
#     for s in range(-h - w, h + w):                      # Cover a wide range of distances to find all points of the line within the image...
#         i, j = int(i0 - s * math.sin(alpha) + 0.5), int(j0 + s * math.cos(alpha) + 0.5)     # ... by going from (i0, j0) in perpendicular direction from the normal
#         if i >= 0 and i < h and j >= 0 and j < w:
#             if s % 20 < 10:                             # Draw black and white dashes for better visibility
#                 color = (0, 0, 0)
#             else:
#                 color = (255, 255, 255)
#             cv2.circle(img, (j, i), 1, color, 2)

In [3]:
# NUM_ALPHA_BINS = 500        # Bins are the cells or "vote counters" for each dimension in the output space
# NUM_D_BINS = 500            # The output space has shape NUM_ALPHA_BINS x NUM_D_BINS
# NUM_MAXIMA = 9              # Number of most significant lines to be found in the input image
# INPUT_IMAGE = 'desk.png'

# orig_img = cv2.imread(INPUT_IMAGE)
# cv2.imshow('Input Image', orig_img)
# cv2.waitKey(1)

In [4]:
# edge_img = compute_edge_image(orig_img)

# cv2.imshow('Edge Image', edge_img)
# cv2.waitKey(1)

In [5]:
# output_space, alpha_bins, d_max = hough_transform_lines(edge_img, NUM_ALPHA_BINS, NUM_D_BINS)
# output_img = (output_space * 255.0 / np.max(output_space)).astype(np.uint8)
# output_max_img = cv2.cvtColor(output_img, cv2.COLOR_GRAY2BGR)       # Convert output space image to color so we can mark maxima with red circles
# output_lines_img = orig_img.copy()
# line_parameters = find_hough_maxima(output_space, NUM_MAXIMA, 40, 40)

# for (alpha, d) in line_parameters:
#     cv2.circle(output_max_img, (d, alpha), 10, (0, 0, 255), 2)
#     draw_hough_line(output_lines_img, alpha_bins[alpha], d * d_max / NUM_D_BINS)

# cv2.imshow('Output Space with Maxima', output_max_img)
# cv2.imshow('Input Image with Lines', output_lines_img)
# cv2.waitKey(0)

In [6]:
from collections import defaultdict

def hough_transform_circles(input_space, min_radius, max_radius, num_row_bins, num_col_bins, num_radius_bins):

    # Thetas is bins created from 0 to 360 degree with increment of the dtheta
    thetas = np.arange(0, 360, step=int(360 / 100))
    
    step = int((max_radius- min_radius)/ num_radius_bins)

    # Radius ranges from r_min to r_max with step according to num_radius_bins
    rs = np.arange(min_radius, max_radius, step=step)

    cos_thetas = np.cos(np.deg2rad(thetas))
    sin_thetas = np.sin(np.deg2rad(thetas))

    # candidate circles dx and dy for different delta radius
    circle_candidates = []
    for r in rs:
        for t in range(num_thetas):
            circle_candidates.append((r, int(r * cos_thetas[t]), int(r * sin_thetas[t])))

    output_space = defaultdict(int)

    for y in range(num_row_bins):
        for x in range(num_col_bins):
            if input_space[y][x] != 0: #white pixel
            # Edge pixel here: vote for circle from the candidate circles passing through this pixel.
                for r, rcos_t, rsin_t in circle_candidates:
                    x_center = x - rcos_t
                    y_center = y - rsin_t
                    output_space[(x_center, y_center, r)] += 1 #vote

    return output_space

def find_hough_maxima_circles(output_space, bin_threshold):
    maxima = []

    # Sort the cacdidates based on the votes
    for candidate_circle, votes in sorted(output_space.items(), key=lambda i: -i[1]):
        x, y, r = candidate_circle
        current_vote_percentage = votes / 100
        if current_vote_percentage >= bin_threshold: 
            # Shortlist the circle for final result
            maxima.append((x, y, r, current_vote_percentage))
            print(x, y, r, current_vote_percentage)
    
    return maxima
            
def draw_hough_circles(output_img, maxima, pixel_threshold):            
    output_img = output_img.copy()
    postprocess_circles = []
    for x, y, r, v in maxima:
      # Remove nearby duplicate circles based on pixel_threshold
      if all(abs(x - xc) > pixel_threshold or abs(y - yc) > pixel_threshold or abs(r - rc) > pixel_threshold for xc, yc, rc, v in postprocess_circles):
        postprocess_circles.append((x, y, r, v))
    maxima = postprocess_circles

    # Draw shortlisted circles on the output image
    for x, y, r, v in maxima:
        output_img = cv2.circle(output_img, (x,y), r, (0,255,0), 2)

    return output_img


In [7]:
num_row_bins = 300        # Bins are the cells or "vote counters" for each dimension in the output space
num_col_bins = 300            # The output space has shape num_row_bins x num_col_bins x ((max_radius- min_radius)/ num_radius_bins)
min_radius = 30
max_radius = 40
num_radius_bins = max_radius - min_radius
delta_r = 1
num_thetas = 100
bin_threshold = 0.9
min_edge_threshold = 100
max_edge_threshold = 200
pixel_threshold = 5
    
INPUT_IMAGE = 'circles.jpg'

orig_img = cv2.imread(INPUT_IMAGE)
cv2.imshow('Input Image', orig_img)
cv2.waitKey(1)
edge_img = compute_edge_image(orig_img)

cv2.imshow('Edge Image', edge_img)
cv2.waitKey(1)

print(edge_img.shape[:2])
# resize_edge_image = cv2.resize(edge_img, (100,100), interpolation = cv2.INTER_AREA)
# print(resize_edge_image.shape[:2])
# cv2.imshow('resize_edge_image Image', resize_edge_image)
cv2.waitKey(1)

(300, 300)


-1

In [8]:
output_space = hough_transform_circles(edge_img, min_radius, max_radius, num_row_bins, num_col_bins, num_radius_bins)


In [9]:
maxima = find_hough_maxima_circles(output_space, bin_threshold)

# resize_og_image = cv2.resize(orig_img, (100,100), interpolation = cv2.INTER_AREA)
# output_circles_img, circles = draw_hough_circles(resize_og_image, maxima, pixel_threshold)
output_circles_img = draw_hough_circles(orig_img, maxima, pixel_threshold)


cv2.imshow('Input Image with Lines', output_circles_img)
cv2.waitKey(0)

64 98 31 1.0
65 98 31 1.0
64 99 31 1.0
65 99 31 1.0
195 100 31 1.0
196 100 31 1.0
197 100 31 1.0
195 100 30 1.0
196 100 30 1.0
196 101 31 1.0
128 191 31 1.0
127 192 31 1.0
128 191 30 1.0
64 99 32 0.99
196 99 31 0.99
196 100 32 0.99
197 99 31 0.99
196 101 32 0.99
195 101 31 0.99
197 101 31 0.99
128 190 31 0.99
129 190 31 0.99
127 191 31 0.99
129 191 31 0.99
128 192 31 0.99
129 191 30 0.99
65 97 32 0.98
64 98 32 0.98
65 98 32 0.98
65 99 32 0.98
195 99 31 0.98
129 192 31 0.98
63 99 31 0.97
65 100 32 0.97
198 100 32 0.97
197 100 32 0.97
197 101 33 0.97
197 101 32 0.97
127 190 31 0.97
127 191 30 0.97
64 97 32 0.96
64 97 31 0.96
64 100 32 0.96
64 100 31 0.96
65 100 31 0.96
196 99 30 0.96
197 100 30 0.96
197 102 32 0.96
195 101 30 0.96
128 191 32 0.96
129 191 32 0.96
128 192 32 0.96
127 192 30 0.96
66 98 32 0.95
66 98 31 0.95
66 99 31 0.95
196 99 32 0.95
195 100 32 0.95
63 100 31 0.95
194 100 31 0.95
198 101 32 0.95
127 191 32 0.95
128 192 30 0.95
63 98 31 0.94
66 99 32 0.94
64 98 30 0.94
197

48